In [1]:
#import libraries
import cvxpy as cp
import numpy as np
import pandas as pd

#load data
xls = pd.ExcelFile('data\case6.xlsx')
branch = pd.read_excel(xls, 'branch')
gen = pd.read_excel(xls, 'gen')
bus = pd.read_excel(xls, 'bus')
cost = pd.read_excel(xls, 'cost')

#Y matrix
base = 100
bus_len = bus.shape[0]
gen_len = len(gen['bus'].ravel())

Ybus = np.matrix(np.zeros([bus_len,bus_len]),dtype=complex)
shunt = np.zeros(bus_len, dtype=complex)
x = [pd.DataFrame() for i in range(bus_len)]
for i in range(bus_len):  
    x[i] = branch.loc[branch['fbus'] == i]
    fro = [j for j in x[i]['tbus']]
    fro = pd.DataFrame(fro)
    for j in range(len(fro)):
        Ybus[i-1,fro.loc[j][0]-1] =  -1/complex(pd.DataFrame(x[i]['r']).iloc[j][0],pd.DataFrame(x[i]['x']).iloc[j][0])
        Ybus[fro.loc[j][0]-1,i-1] =  -1/complex(pd.DataFrame(x[i]['r']).iloc[j][0],pd.DataFrame(x[i]['x']).iloc[j][0])

for i in range(bus_len):
    Ybus[i,i] = np.sum(Ybus[i,:])*(-1) + complex(0,sum(branch.loc[(branch['fbus']==i+1) | (branch['tbus']==i+1)]['b'])/2)

W = cp.Variable((2*bus_len, 2*bus_len), PSD=True)

e = ['' for i in range(bus_len)]
ee= ['' for i in range(bus_len)]
for i in range(bus_len):
    e[i] = np.matrix(np.zeros(bus_len))
    e[i][0,i] = 1
    ee[i] = np.transpose(e[i])*e[i]


l = [['' for i in range(len(bus))] for i in range(len(bus))]
L = [['' for i in range(len(bus))] for i in range(len(bus))]
L_ = [['' for i in range(len(bus))] for i in range(len(bus))]
M = ['' for i in range(len(bus))]
N = ['' for i in range(len(bus))]
p = ['' for i in range(len(bus))]
q = ['' for i in range(len(bus))]
for i in range(len(bus)):
    M[i] = np.concatenate([np.concatenate([ee[i],np.matrix(np.zeros([bus_len,bus_len]))],axis=1),
               np.concatenate([np.matrix(np.zeros([bus_len,bus_len])),ee[i]],axis=1)])
    N[i] = np.concatenate([np.concatenate([np.matrix(np.zeros([bus_len,bus_len])),np.matrix(np.zeros([bus_len,bus_len]))],axis=1),
               np.concatenate([np.matrix(np.zeros([bus_len,bus_len])),ee[i]],axis=1)])
    p[i] = 0.5*np.concatenate([np.concatenate([np.real(np.transpose(ee[i]*Ybus) + ee[i]*Ybus),np.imag(np.transpose(ee[i]*Ybus) - ee[i]*Ybus)],axis=1),
                             np.concatenate([np.imag(ee[i]*Ybus - np.transpose(ee[i]*Ybus)), np.real(np.transpose(ee[i]*Ybus) + ee[i]*Ybus)],axis=1)])
    q[i] = -0.5*np.concatenate([np.concatenate([np.imag(np.transpose(ee[i]*Ybus) + ee[i]*Ybus),np.real(ee[i]*Ybus - np.transpose(ee[i]*Ybus))],axis=1),
                             np.concatenate([np.real(np.transpose(ee[i]*Ybus) - ee[i]*Ybus), np.imag(np.transpose(ee[i]*Ybus) + ee[i]*Ybus)],axis=1)])

for i in range(len(branch)):
    f = int(branch.iloc[i]['fbus'])
    t = int(branch.iloc[i]['tbus'])
    r = branch.loc[(branch['fbus']==f) & ((branch['tbus']==t))]['r'][i]
    x = branch.loc[(branch['fbus']==f) & ((branch['tbus']==t))]['x'][i]
    l[f-1][t-1] = (complex(0,branch.loc[(branch['fbus']==f) & ((branch['tbus']==t))]['b'][i])/2 + 1/(complex(r,x)))*np.transpose(e[f-1])*e[f-1] - 1/(complex(r,x))*np.transpose(e[f-1])*e[t-1]
    l[t-1][f-1] = (complex(0,branch.loc[(branch['fbus']==f) & ((branch['tbus']==t))]['b'][i])/2 + 1/(complex(r,x)))*np.transpose(e[t-1])*e[t-1] - 1/(complex(r,x))*np.transpose(e[t-1])*e[f-1]
    L_[f-1][t-1] = -0.5*np.concatenate([np.concatenate([np.imag(l[f-1][t-1] + np.transpose(l[f-1][t-1])),np.real(-np.transpose(l[f-1][t-1]) + l[f-1][t-1])],axis=1),
                             np.concatenate([np.real(-l[f-1][t-1] + np.transpose(l[f-1][t-1])), np.imag(l[f-1][t-1] + np.transpose(l[f-1][t-1]))],axis=1)])
    L_[t-1][f-1] = -0.5*np.concatenate([np.concatenate([np.imag(l[t-1][f-1] + np.transpose(l[t-1][f-1])),np.real(-np.transpose(l[t-1][f-1]) + l[t-1][f-1])],axis=1),
                             np.concatenate([np.real(-l[t-1][f-1] + np.transpose(l[t-1][f-1])), np.imag(l[t-1][f-1] + np.transpose(l[t-1][f-1]))],axis=1)])
    L[f-1][t-1] = 0.5*np.concatenate([np.concatenate([np.real(l[f-1][t-1] + np.transpose(l[f-1][t-1])),np.imag(np.transpose(l[f-1][t-1]) - l[f-1][t-1])],axis=1),
                             np.concatenate([np.imag(l[f-1][t-1] - np.transpose(l[f-1][t-1])), np.real(l[f-1][t-1] + np.transpose(l[f-1][t-1]))],axis=1)])
    L[t-1][f-1] = 0.5*np.concatenate([np.concatenate([np.real(l[t-1][f-1] + np.transpose(l[t-1][f-1])),np.imag(np.transpose(l[t-1][f-1]) - l[t-1][f-1])],axis=1),
                             np.concatenate([np.imag(l[t-1][f-1] - np.transpose(l[t-1][f-1])), np.real(l[t-1][f-1] + np.transpose(l[t-1][f-1]))],axis=1)])
    
# declare variables
Pd = np.zeros(bus_len)
Qd = np.zeros(bus_len)
Vmax = np.zeros(bus_len)
Vmin = np.zeros(bus_len)
Pmax = np.zeros(bus_len)
Pmin = np.zeros(bus_len)
Qmax = np.zeros(bus_len)
Qmin = np.zeros(bus_len)
cost_lin = np.zeros(bus_len)
cost_quad = np.zeros(bus_len)

#load data into variables
for i in range(bus_len):
    Pd[i] = bus['Pd'][i]/base
    Qd[i] = bus['Qd'][i]/base
    Vmax[i] = bus['Vmax'][i]
    Vmin[i] = bus['Vmin'][i]

for i in range(gen_len):
    k = i
    i = gen['bus'].ravel()[i]
    Pmax[i-1] = gen.loc[gen['bus'] == i]['Pmax'].ravel()[0]/base
    Pmin[i-1] = gen.loc[gen['bus'] == i]['Pmin'].ravel()[0]/base
    Qmax[i-1] = gen.loc[gen['bus'] == i]['Qmax'].ravel()[0]/base
    Qmin[i-1] = gen.loc[gen['bus'] == i]['Qmin'].ravel()[0]/base
    cost_lin[i-1] = cost['lin'][k]*base
    cost_quad[i-1] = cost['quad'][k]*base**2
    
Pg = [0 for i in range(bus_len)]
Qg = [0 for i in range(bus_len)]
for i in range(gen_len):
    Pg[gen['bus'][i]-1] = ""
    Qg[gen['bus'][i]-1] = ""
    Pg[gen['bus'][i]-1] = cp.trace(p[gen['bus'][i]-1]@W) + Pd[gen['bus'][i]-1]
    Qg[gen['bus'][i]-1] = cp.trace(q[gen['bus'][i]-1]@W) + Qd[gen['bus'][i]-1]


# Pg = ["" for i in range(gen_len)]
# Qg = ["" for i in range(gen_len)]
# for i in range(gen_len):
#     Pg[i] = cp.trace(p[gen['bus'][i]-1]@W) + Pd[gen['bus'][i]-1]
#     Qg[i] = cp.trace(q[gen['bus'][i]-1]@W) + Qd[gen['bus'][i]-1]
    
# Cost function
# objective = cp.Minimize(sum(cost['cons']) + 
#     (sum(Pg[i]*cost_lin[i] for i in range(bus_len))) + 
#     (sum(cost_quad[i]*(Pg[i]**2) for i in range(bus_len)))
#                        )

# objective = cp.Minimize(sum([Pg[i] for i in range(bus_len)])
#                        )

#objective = cp.Minimize(sum(Pg))

objective = cp.Minimize(sum([Pg[i] for i in range(bus_len)]) + sum([Qg[i] for i in range(bus_len)])
                       )


#Constraints
constraints = [(W >> 0)]
constraints += [W[:,bus_len] == 0]
constraints += [W[bus_len,:] == 0]


#Active power balance, Reactive power balance, bus voltages constraints
# for i in range(bus_len):
#     constraints += [cp.trace(M[i]@W) <= Vmax[i]**2]
#     constraints += [cp.trace(M[i]@W) >= Vmin[i]**2]
#     if(Pmax[i]-Pd[i] > Pmin[i]-Pd[i]):
#         constraints += [cp.trace(p[i]@W) <= Pmax[i]-Pd[i]]
#         constraints += [cp.trace(p[i]@W) >= Pmin[i]-Pd[i]]
#     elif(Pmax[i]-Pd[i] == Pmax[i]-Pd[i]):
#         constraints += [cp.trace(p[i]@W) == Pmax[i]-Pd[i]]
#     else:
#         print("Error")


#     if(Qmax[i]-Qd[i] > Qmin[i]-Qd[i]):
#         constraints += [cp.trace(q[i]@W) <= Qmax[i]-Qd[i]]
#         constraints += [cp.trace(q[i]@W) >= Qmin[i]-Qd[i]]
#     elif(Qmax[i]-Qd[i] == Qmin[i]-Qd[i]):
#         constraints += [cp.trace(q[i]@W) == Qmax[i]-Qd[i]]
#     else:
#         print("Error")
    
for i in range(bus_len):
    constraints += [cp.trace(M[i]@W) <= Vmax[i]**2]
    constraints += [cp.trace(M[i]@W) >= Vmin[i]**2]
    constraints += [cp.trace(p[i]@W) <= Pmax[i]-Pd[i]]
    constraints += [cp.trace(p[i]@W) >= Pmin[i]-Pd[i]]
    constraints += [cp.trace(q[i]@W) <= Qmax[i]-Qd[i]]
    constraints += [cp.trace(q[i]@W) >= Qmin[i]-Qd[i]]
 
#Flow constraints   
for i in range(len(branch)):
    f = int(branch.iloc[i]['fbus'])
    t = int(branch.iloc[i]['tbus']) 
    constraints += [cp.trace(L[f-1][t-1]@W)**2 + cp.trace(L_[f-1][t-1]@W)**2 <= (branch.loc[i]['rateA']/base)**2]
    constraints += [cp.trace(L[t-1][f-1]@W)**2 + cp.trace(L_[t-1][f-1]@W)**2 <= (branch.loc[i]['rateA']/base)**2]
    
    
#Objective function
prob = cp.Problem(objective, constraints)

# solve
prob.solve(solver=cp.MOSEK)#, verbose=True)

#Extract voltage phasors from W matrix
[evl,evc] = np.linalg.eigh(W.value)
eigWratio = evl[-1]/evl[-2]
Vopt = np.sqrt(evl[-1])*evc[:,-1]
Vopt = Vopt[0:bus_len] + [complex(0,Vopt[i]) for i in range(bus_len,2*bus_len)]
abs(np.around(Vopt,decimals=4))

array([1.07      , 1.07001572, 1.07000864, 1.04406139, 1.03903819,
       1.0498839 ])

In [2]:
#Store values from convex relaxation
vv = np.zeros(bus_len)
qv = np.zeros(bus_len)
pv = np.zeros(bus_len)
for i in range(bus_len):
    vv[i] = abs(np.around(Vopt,decimals=4))[i]
    if(i<gen_len):
        pv[i] = Pg[i].value
        qv[i] = Qg[i].value

In [3]:
#Exploring the gap between global optimum in Relaxed space and local optimum in ACOPF
#6 bus system

#import libraries
import cmath
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.environ import *

#load data
xls = pd.ExcelFile('data\case6.xlsx')
branch = pd.read_excel(xls, 'branch')
gen = pd.read_excel(xls, 'gen')
bus = pd.read_excel(xls, 'bus')
cost = pd.read_excel(xls, 'cost')

#Y matrix
base = 100
bus_len = bus.shape[0]
gen_len = len(gen['bus'].ravel())

Ybus = np.matrix(np.zeros([bus_len,bus_len]),dtype=complex)
shunt = np.zeros(bus_len, dtype=complex)
x = [pd.DataFrame() for i in range(bus_len)]
for i in range(bus_len):  
    x[i] = branch.loc[branch['fbus'] == i]
    fro = [j for j in x[i]['tbus']]
    fro = pd.DataFrame(fro)
    for j in range(len(fro)):
        Ybus[i-1,fro.loc[j][0]-1] =  -1/complex(pd.DataFrame(x[i]['r']).iloc[j][0],pd.DataFrame(x[i]['x']).iloc[j][0])
        Ybus[fro.loc[j][0]-1,i-1] =  -1/complex(pd.DataFrame(x[i]['r']).iloc[j][0],pd.DataFrame(x[i]['x']).iloc[j][0])
        shunt[j] = complex(0,sum(x[i]['b'])/2)

for i in range(bus_len):
    Ybus[i,i] = np.sum(Ybus[i,:])*(-1) + complex(0,sum(branch.loc[(branch['fbus']==i+1) | (branch['tbus']==i+1)]['b'])/2)


# declare variables
Pd = np.zeros(bus_len)
Qd = np.zeros(bus_len)
Vmax = np.zeros(bus_len)
Vmin = np.zeros(bus_len)
Pmax = np.zeros(bus_len)
Pmin = np.zeros(bus_len)
Qmax = np.zeros(bus_len)
Qmin = np.zeros(bus_len)
cost_lin = np.zeros(bus_len)
cost_quad = np.zeros(bus_len)

for i in range(bus_len):
    Pd[i] = bus['Pd'][i]/base
    Qd[i] = bus['Qd'][i]/base
    Vmax[i] = bus['Vmax'][i]
    Vmin[i] = bus['Vmin'][i]

for i in range(gen_len):
    k = i
    i = gen['bus'].ravel()[i]
    Pmax[i-1] = gen.loc[gen['bus'] == i]['Pmax'].ravel()[0]/base
    Pmin[i-1] = gen.loc[gen['bus'] == i]['Pmin'].ravel()[0]/base
    Qmax[i-1] = gen.loc[gen['bus'] == i]['Qmax'].ravel()[0]/base
    Qmin[i-1] = gen.loc[gen['bus'] == i]['Qmin'].ravel()[0]/base
    cost_lin[i-1] = cost['lin'][k]*base
    cost_quad[i-1] = cost['quad'][k]*base**2
      
#Create a model
model = pyo.ConcreteModel()

model.IDX1 = range(gen_len)
model.IDX2 = range(bus_len)
model.Pg = pyo.Var(model.IDX2)
model.Qg = pyo.Var(model.IDX2)
model.t = pyo.Var(model.IDX2)
model.v = pyo.Var(model.IDX2)


for i in model.IDX2:
    model.Pg[i].fix(0) 
    model.Qg[i].fix(0) 
    
for i in model.IDX1:
    model.Pg[gen['bus'][i]-1].fixed = False
    model.Qg[gen['bus'][i]-1].fixed = False


model.d1 = pyo.Var(model.IDX2)
model.d2 = pyo.Var(model.IDX2)
model.d3 = pyo.Var(model.IDX2)

# declare constraints
model.c = pyo.ConstraintList()
#model.c.add(expr = sum(model.Pg[i] for i in model.IDX2) == sum(Pd))
for i in range(bus_len):
    model.c.add(expr = pv[i]+model.d1[i] <= Pmax[i])
    model.c.add(expr = pv[i]+model.d1[i] >= Pmin[i])
    model.c.add(expr = qv[i]+model.d2[i] <= Qmax[i])
    model.c.add(expr = qv[i]+model.d2[i] >= Qmin[i])
    model.c.add(expr = vv[i]+model.d3[i] <= Vmax[i])
    model.c.add(expr = vv[i]+model.d3[i] >= Vmin[i])    

#Bus voltage angle initial condition
model.c.add(model.t[0] == 0)

#Nodal equations
for i in range(bus_len):
    model.c.add(expr = sum([(vv[i]+model.d3[i])*(vv[j]+model.d3[j])*cmath.polar(Ybus[i,j])[0]*cos(model.t[i]-model.t[j]-cmath.polar(Ybus[i,j])[1]) for j in range(bus_len)]) - model.Pg[i] + Pd[i]== 0)
    model.c.add(expr = sum([(vv[i]+model.d3[i])*(vv[j]+model.d3[j])*cmath.polar(Ybus[i,j])[0]*sin(model.t[i]-model.t[j]-cmath.polar(Ybus[i,j])[1]) for j in range(bus_len)]) - model.Qg[i] + Qd[i]== 0)

#Line flow constraints
for i in range(len(branch)): 
    x = int(branch.loc[i]['fbus'])-1
    y = int(branch.loc[i]['tbus'])-1
    model.c.add(expr = abs(((model.v[x]**2)*cmath.polar(Ybus[x,y])[0])*cos(cmath.polar(Ybus[x,y])[1]) - model.v[x]*model.v[y]*cmath.polar(Ybus[x,y])[0]*cos(model.t[x]-model.t[y]-cmath.polar(Ybus[x,y])[1])) <= int(branch.loc[i]['rateA'])/base)
      
# declare objective
model.cost = pyo.Objective( 
    expr = sum([model.d1[i]**2 for i in range(bus_len)]) + sum([model.d2[i]**2 for i in range(bus_len)]) + sum([model.d3[i]**2 for i in range(bus_len)]) 
) 

# solve
SolverFactory('ipopt').solve(model).write()


#Print results
print('Cost = ', model.cost())




# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 60
  Number of variables: 36
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.04148697853088379
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
Cost =